In [0]:
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import scipy as sp

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

import os
import sys
import gc
import copy

import tqdm
import itertools

import subprocess

In [0]:
#baseDir='./'
!rm -r sample_data

If you are viewing this in google colab, you will need to clone the repository first.
To do so uncomment the two code cells below

In [3]:
!git clone https://github.com/wesleymsmith/Piezo_PIP2_binding_analysis.git

Cloning into 'Piezo_PIP2_binding_analysis'...
remote: Enumerating objects: 1528, done.
remote: Counting objects: 100% (1528/1528), done.
remote: Compressing objects: 100% (254/254), done.
remote: Total 5087 (delta 1249), reused 1520 (delta 1241), pack-reused 3559
Receiving objects: 100% (5087/5087), 172.88 MiB | 21.76 MiB/s, done.
Resolving deltas: 100% (3981/3981), done.
Checking out files: 100% (6644/6644), done.


In [0]:
baseDir='Piezo_PIP2_binding_analysis/'

In [0]:
xcelData=pd.read_excel(baseDir+'Residue_ID_total_occupancy_10_1_2019.xlsx',
              sheet_name=None)

In [6]:
@interact
def show_data(sheet_name=xcelData.keys()):
    return xcelData[sheet_name]

interactive(children=(Dropdown(description=u'sheet_name', options=(u'cg', u'Sheet1', u'aa', u'tension_30ns', u…

In [7]:
#the above tables are
#aa - result summary for all atom simulation
#cg - result summary for coarse grain simulation
#tension_30ns - results of all atom simulation with membrane tension
#sheet 1 is apparently blank...
#resinfo_table - mapping between cryo-em structure sequence and all atom residue ids
resinfoDataSheet=xcelData['resinfo_table']
resinfoTable=resinfoDataSheet[
    resinfoDataSheet.columns[[0,3,5,7]]][2:]
resinfoTable.columns=['PDB_ID','Arm1_Resid','Arm2_Resid','Arm3_Resid']
resinfoTable.head()

,PDB_ID,Arm1_Resid,Arm2_Resid,Arm3_Resid
2,782,1,1419,2837
3,783,2,1420,2838
4,784,3,1421,2839
5,785,4,1422,2840
6,786,5,1423,2841


Before continuing, lets check to see if we have any missing values.

In [0]:
for colName in resinfoTable.columns:
    print '%s:'%colName,
    print resinfoTable[colName].isna().sum()

PDB_ID: 53
Arm1_Resid: 0
Arm2_Resid: 0
Arm3_Resid: 0


In [0]:
resinfoTable[resinfoTable['Arm1_Resid']==1329]

,PDB_ID,Arm1_Resid,Arm2_Resid,Arm3_Resid
1330,NaN,1329,2747,4165


In [0]:
np.isnan(resinfoTableFilled.PDB_ID[97])

NameError: name 'resinfoTableFilled' is not defined

After our inspection above, we see there is a problem with trying to directly use the resinfo table above.
Specifically, the 'PDB_ID' column we want to use has missing values!

Fortunately, we know that these missing values are gaps in the pdb sequence for structures that could not be resolved in the cryo-em. The corresponding sequence should, therefore, increase linearly across these gaps.

This makes our imputation strategy relatively straight forward. We will iterate over the PDB_ID column. And keep track of the last value we see. When we find valid (integer) values, we just update the last value variable. If we see a missing value we simply increment that last value variable then set the missing entry to be equal to that value.

In [8]:
resinfoTableFilled=copy.deepcopy(resinfoTable)
print resinfoTableFilled.PDB_ID.isna().sum()
for iEntry in resinfoTableFilled.PDB_ID.index:
    entry=resinfoTableFilled.PDB_ID[iEntry]
    if np.isnan(entry):
        lastVal=lastVal+1
        resinfoTableFilled.PDB_ID[iEntry]=lastVal
    else:
        lastVal=entry
resinfoTableFilled.PDB_ID.isna().sum()

53


0

In [9]:
#the pdb residue id's are sequential, but have gaps corresponding
#to unresolved amino acids in the Cryo-EM structure.
#The easy solution is just to fill in linearly.
#The three arms in our simulation structure are identical, so
#we can generate our back-map by just repeating the pdb sequence 3 times
simResid_to_pdbResid=list(np.array(resinfoTableFilled.PDB_ID))*3
print ', '.join(map(str,simResid_to_pdbResid))

782, 783, 784, 785, 786, 787, 788, 789, 790, 791, 792, 793, 794, 795, 796, 797, 798, 799, 800, 801, 802, 803, 804, 805, 806, 807, 808, 809, 810, 811, 812, 813, 814, 815, 816, 817, 818, 819, 820, 821, 822, 823, 824, 825, 826, 827, 828, 829, 830, 831, 832, 833, 834, 835, 836, 837, 838, 839, 840, 841, 842, 843, 844, 845, 846, 847, 848, 849, 850, 851, 852, 853, 854, 855, 856, 857, 858, 859, 860, 861, 862, 863, 864, 865, 866, 867, 868, 869, 870, 871, 872, 873, 874, 875, 876, 877, 878, 879, 880, 881, 882, 883, 884, 885, 886, 887, 888, 889, 890, 891, 892, 893, 894, 895, 896, 897, 898, 899, 900, 901, 902, 903, 904, 905, 906, 907, 908, 909, 910, 911, 912, 913, 914, 915, 916, 917, 918, 919, 920, 921, 922, 923, 924, 925, 926, 927, 928, 929, 930, 931, 932, 933, 934, 935, 936, 937, 938, 939, 940, 941, 942, 943, 944, 945, 946, 947, 948, 949, 950, 951, 952, 953, 954, 955, 956, 957, 958, 959, 960, 961, 962, 963, 964, 965, 966, 967, 968, 969, 970, 971, 972, 973, 974, 975, 976, 977, 978, 979, 980, 981, 

While the above excel sheet provides a useful summary at a glance, we would like to have direct access to the distribution of residence times rather than just the mean max and cummulative sum over all lipids. 

Using the set of all individually measured residence times, we can fit a model distribution. More specifically, the reciporical of residence time would correspond to a frequency. Specifically, the reciporical of residence time gives us the corresponding unbinding frequency. 

This can then be used to fit an appropriate distribution (geometric distribution would be one choice) and provide a characteristic unbinding frequency (or characteristic residence time as its reciporical). More over, it can give us a bound / confidence interval of this distribution as well.

We can then repeat this process for the all atom model. While CG is expected to have shorter residence times due to the notably lower membrane viscocity, we should still be able to see if the ranking and / or relative characteristic residence time / unbinding frequencies match (for each protein amino acid). If CG can rank amino acids in the correct order, based upon PIP2 residence times, then we can be confident that it is functioning well as a model for correctly predicting lipid binding sites.

Below, this residence time distribution data can be exctracted from the coarse grain simulation data files which list individual PIP2 residence time observations for each protein residue (amino acid)

in the Raw_PIP2_CG_residence_time_data directory, the .xvg files contain
the 'occupancy' of each amino acid at each output time step.
The occupancy is zero if there were no PIP2 lipids in contact
and non-zero if there was at least one PIP2 lipid in contact.
The first step is to extract these individual timeseries
into a joint table.

In [10]:
#os.listdir can be used to generate a list of all xvg files present
!cg_RawData_dir=baseDir+'Raw_PIP2_CG_residence_time_data/'
cg_RawData_dir=baseDir+'Raw_POPS_data/calc_CG_all_lysArg/'
cg_dataFile_list=[dataFileName for dataFileName in os.listdir(cg_RawData_dir) \
             if 'xvg' in dataFileName]
cg_dataFile_list

['id_3677_mask.xvg',
 'id_1190_mask.xvg',
 'id_2857_mask.xvg',
 'id_3481_mask.xvg',
 'id_3526_mask.xvg',
 'id_443_mask.xvg',
 'id_3373_mask.xvg',
 'id_2619_mask.xvg',
 'id_2178_mask.xvg',
 'id_3821_mask.xvg',
 'id_3123_mask.xvg',
 'id_3529_mask.xvg',
 'id_2062_mask.xvg',
 'id_2195_mask.xvg',
 'id_1553_mask.xvg',
 'id_913_mask.xvg',
 'id_1995_mask.xvg',
 'id_3679_mask.xvg',
 'id_3022_mask.xvg',
 'id_1625_mask.xvg',
 'id_985_mask.xvg',
 'id_2925_mask.xvg',
 'id_3170_mask.xvg',
 'id_577_mask.xvg',
 'id_15_mask.xvg',
 'id_2394_mask.xvg',
 'id_1103_mask.xvg',
 'id_3349_mask.xvg',
 'id_3881_mask.xvg',
 'id_420_mask.xvg',
 'id_1984_mask.xvg',
 'id_4222_mask.xvg',
 'id_3322_mask.xvg',
 'id_667_mask.xvg',
 'id_2475_mask.xvg',
 'id_743_mask.xvg',
 'id_1668_mask.xvg',
 'id_3614_mask.xvg',
 'id_976_mask.xvg',
 'id_3754_mask.xvg',
 'id_2837_mask.xvg',
 'id_1501_mask.xvg',
 'id_1056_mask.xvg',
 'id_3256_mask.xvg',
 'id_2742_mask.xvg',
 'id_1906_mask.xvg',
 'id_559_mask.xvg',
 'id_373_mask.xvg',
 'id

In [30]:
#Each of the above files contains an 18 line header, followed by 2 column occupancy series
#the data can be easily extracted using pd.read_table
cgDataTables=[]
for dataFileName in tqdm.tqdm_notebook(cg_dataFile_list):
    resID=int(dataFileName.split('_')[1])
    seqID=simResid_to_pdbResid[resID-1]
    tempTable=pd.read_csv(
        cg_RawData_dir+dataFileName,
        skiprows=17,names=['Time','Occupancy'],
        delim_whitespace=True)
    tempTable['ResID']=resID
    tempTable['SeqID']=seqID
    tempTable['Frame']=np.arange(len(tempTable))
    tempTable=tempTable[['ResID','SeqID','Frame','Time','Occupancy']]
    cgDataTables.append(copy.deepcopy(tempTable))
    
cg_occupancy_data=pd.concat(cgDataTables)
cg_occupancy_data.to_csv(baseDir+"Coarse_Grain_POPS_Occupancy_Data.csv",index=False)
print cg_occupancy_data.head()
print cg_occupancy_data.tail()


   ResID  SeqID  Frame    Time  Occupancy
0   3677   1969      0     0.0          0
1   3677   1969      1  1000.0          0
2   3677   1969      2  2000.0          0
3   3677   1969      3  3000.0          0
4   3677   1969      4  4000.0          0
       ResID  SeqID  Frame        Time  Occupancy
11996   1838   1201  11996  11996000.0          0
11997   1838   1201  11997  11997000.0          0
11998   1838   1201  11998  11998000.0          0
11999   1838   1201  11999  11999000.0          0
12000   1838   1201  12000  12000000.0          0


In [25]:
residtmp=pd.DataFrame(cg_occupancy_data.ResID.unique())
print residtmp.count()
print "......"
segidtmp=pd.DataFrame(cg_occupancy_data.SeqID.unique())
print segidtmp.count()
print residtmp.count()/segidtmp.count()


0    429
dtype: int64
......
0    143
dtype: int64
0    3.0
dtype: float64


In [31]:
#The coarse grain data table is huge, so we need to split it into chunks

os.system("split -l 100000 "+\
          baseDir+"Coarse_Grain_POPS_Occupancy_Data.csv "+\
          baseDir+"Raw_POPS_data/Coarse_Grain_POPS_Occupancy_Data.chunk.")
os.system("rm "+baseDir+"Coarse_Grain_POPS_Occupancy_Data.csv")

0

Next, lets collect the occupancy data for the all atom simulation into a single data frame.

In [47]:
aa_RawData_dir=baseDir+'Raw_POPS_data/calc_aa_190ns_lysArg/'
aa_dataFile_list=[dataFileName for dataFileName in os.listdir(aa_RawData_dir) \
             if 'xvg' in dataFileName]
aa_dataFile_list

#Each of the above files contains an 18 line header, followed by 2 column occupancy series
#the data can be easily extracted using pd.read_table
aaDataTables=[]
for dataFileName in tqdm.tqdm_notebook(aa_dataFile_list):
    resID=int(dataFileName.split('_')[1])
    seqID=simResid_to_pdbResid[resID-1]
    tempTable=pd.read_csv(
        aa_RawData_dir+dataFileName,
        skiprows=17,names=['Time','Occupancy'],
        delim_whitespace=True)
    tempTable['ResID']=resID
    tempTable['SeqID']=seqID
    tempTable['Frame']=np.arange(len(tempTable))
    tempTable=tempTable[['ResID','SeqID','Frame','Time','Occupancy']]
    aaDataTables.append(copy.deepcopy(tempTable))
    
aa_occupancy_data=pd.concat(aaDataTables)
aa_occupancy_data.to_csv(baseDir+"All_Atom_POPS_Occupancy_Data.190ns_0.12ns.csv",index=False)
print aa_occupancy_data.head()
print aa_occupancy_data.tail()


   ResID  SeqID  Frame  Time  Occupancy
0   3677   1969      0   0.0          0
1   3677   1969      1   1.0          0
2   3677   1969      2   2.0          0
3   3677   1969      3   3.0          0
4   3677   1969      4   4.0          0
      ResID  SeqID  Frame    Time  Occupancy
1587   1838   1201   1587  1587.0          1
1588   1838   1201   1588  1588.0          1
1589   1838   1201   1589  1589.0          1
1590   1838   1201   1590  1590.0          1
1591   1838   1201   1591  1591.0          1


In [48]:
aa_occupancy_data.Time.max()*0.12

190.92

In [49]:
residtmp=pd.DataFrame(aa_occupancy_data.ResID.unique())
print aa_occupancy_data.ResID.count()
print residtmp.count()
print "......"
segidtmp=pd.DataFrame(aa_occupancy_data.SeqID.unique())
print segidtmp.count()
print residtmp.count()/segidtmp.count()

682968
0    429
dtype: int64
......
0    143
dtype: int64
0    3.0
dtype: float64


In [51]:
aa_RawData_dir=baseDir+'stretch50ns_anton2_68mNm/'
aa_dataFile_list=[dataFileName for dataFileName in os.listdir(aa_RawData_dir) \
             if 'xvg' in dataFileName]
aa_dataFile_list

#Each of the above files contains an 18 line header, followed by 2 column occupancy series
#the data can be easily extracted using pd.read_table
aaDataTables=[]
tempTable=[]
aa_occupancy_data=[]
for dataFileName in tqdm.tqdm_notebook(aa_dataFile_list):
    resID=int(dataFileName.split('_')[1])
    seqID=simResid_to_pdbResid[resID-1]
    tempTable=pd.read_csv(
        aa_RawData_dir+dataFileName,
        skiprows=17,names=['Time','Occupancy'],
        delim_whitespace=True)
    tempTable['ResID']=resID
    tempTable['SeqID']=seqID
    tempTable['Frame']=np.arange(len(tempTable))
    tempTable=tempTable[['ResID','SeqID','Frame','Time','Occupancy']]
    aaDataTables.append(copy.deepcopy(tempTable))
    
aa_occupancy_data=pd.concat(aaDataTables)
aa_occupancy_data.to_csv(baseDir+"All_Atom_PIP2_Occupancy_Data.stretch50ns_0.12ns.csv",index=False)
print aa_occupancy_data.head()
print aa_occupancy_data.tail()
print aa_occupancy_data.Time.max()*0.12
residtmp=pd.DataFrame(aa_occupancy_data.ResID.unique())
print aa_occupancy_data.ResID.count()/aa_occupancy_data.Frame.max()
print residtmp.count()
print "......"
segidtmp=pd.DataFrame(aa_occupancy_data.SeqID.unique())
print segidtmp.count()
print residtmp.count()/segidtmp.count()


   ResID  SeqID  Frame  Time  Occupancy
0   3677   1969      0   0.0          0
1   3677   1969      1   1.0          0
2   3677   1969      2   2.0          0
3   3677   1969      3   3.0          0
4   3677   1969      4   4.0          0
     ResID  SeqID  Frame   Time  Occupancy
413   1838   1201    413  413.0          0
414   1838   1201    414  414.0          0
415   1838   1201    415  415.0          0
416   1838   1201    416  416.0          0
417   1838   1201    417  417.0          0
50.04
430.0287769784173
0    429
dtype: int64
......
0    143
dtype: int64
0    3.0
dtype: float64


In [54]:
aa_RawData_dir=baseDir+'Raw_PIP2_AA2us_residence_time_data/'
aa_dataFile_list=[dataFileName for dataFileName in os.listdir(aa_RawData_dir) \
             if 'xvg' in dataFileName]
aa_dataFile_list

#Each of the above files contains an 18 line header, followed by 2 column occupancy series
#the data can be easily extracted using pd.read_table
aaDataTables=[]
tempTable=[]
aa_occupancy_data=[]
for dataFileName in tqdm.tqdm_notebook(aa_dataFile_list):
    resID=int(dataFileName.split('_')[1])
    seqID=simResid_to_pdbResid[resID-1]
    tempTable=pd.read_csv(
        aa_RawData_dir+dataFileName,
        skiprows=17,names=['Time','Occupancy'],
        delim_whitespace=True)
    tempTable['ResID']=resID
    tempTable['SeqID']=seqID
    tempTable['Frame']=np.arange(len(tempTable))
    tempTable=tempTable[['ResID','SeqID','Frame','Time','Occupancy']]
    aaDataTables.append(copy.deepcopy(tempTable))
    
aa_occupancy_data=pd.concat(aaDataTables)
aa_occupancy_data.to_csv(baseDir+"All_Atom_PIP2_Occupancy_Data.2us_0.6ns.csv",index=False)
print aa_occupancy_data.head()
print aa_occupancy_data.tail()
print aa_occupancy_data.Time.max()*0.6
residtmp=pd.DataFrame(aa_occupancy_data.ResID.unique())
print aa_occupancy_data.ResID.count()/aa_occupancy_data.Frame.max()
print residtmp.count()
print "......"
segidtmp=pd.DataFrame(aa_occupancy_data.SeqID.unique())
print segidtmp.count()
print residtmp.count()/segidtmp.count()


   ResID  SeqID  Frame  Time  Occupancy
0   3677   1969      0   0.0          0
1   3677   1969      1   5.0          0
2   3677   1969      2  10.0          0
3   3677   1969      3  15.0          0
4   3677   1969      4  20.0          0
      ResID  SeqID  Frame     Time  Occupancy
3329   1838   1201   3329  16645.0          0
3330   1838   1201   3330  16650.0          0
3331   1838   1201   3331  16655.0          0
3332   1838   1201   3332  16660.0          0
3333   1838   1201   3333  16665.0          0
9999.0
429
0    429
dtype: int64
......
0    143
dtype: int64
0    3.0
dtype: float64
